<h1><center>Laboratorio 4: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Constanza Peña
- Nombre de alumno 2: Benjamín Tejeda


### **Link de repositorio de GitHub:** `https://github.com/constanzaps/laboratorios_LDS`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Clasificación con texto.
- Clasificación en `scikit-learn`.
- Modelos a través del uso de `pipeline`.
- Optimización de modelos usando `GridSearchCV`.

## Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar las ventajas que nos ofrece crear un pipeline.
- Obtener caracteristicas desde texto.
- Crear modelos de clasificación de texto.
- Optimizar la clasificación de texto usando wordclouds.
- Usar herramientas de visualización de texto como las wordclouds.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


     |████████████████████████████████| 13.2MB 202kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 81kB 2.6MB/s 
     |████████████████████████████████| 1.2MB 15.0MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=2ebf64187dbdcb64d3b3cfa8d4dfe9f55db53517a5d54ad0c0c7b670d370af19
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.2-cp37-none-any.whl size=51362 sha256=b39ccbc5c4303e738e469921c5068b9269005dba76dd1ef74289a4f447bf2c78
  Stored in directory: /root/.cache/pip/wheels/ba/52/4e/4c28d04d144a28f89e2575fb63628df6e6d49b56c5ddd0c74e
Successfully built umap-learn pynndescent
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package s

True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://static.wikia.nocookie.net/p__/images/a/a2/Bat-Cow.jpg/revision/latest?cb=20180108185037&path-prefix=protagonist" width="350">
</p>

En vez de estar oprotunamente desarrollando las tareas y las evaluaciones correspondientes al curso, su profesor de catedra y su auxiliar discuten acerca la alineación del personaje de ficción *Bat-Cow*. 

El cuerpo docente, no logra ponerse de acuerdo acerca de la alineación del personaje, es decir, si lucha junto a las fuerzas del bien, si neutral a cualquier eventualidad o derechamente es un villano.
El auxiliar plantea (de forma superficial) que *Bat-cow* posee una siniestra mirada, común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor (*se cree filósofo... y*) plantea que tal como los humanos no nacen malos, no existe motivo por el cual un rumiante humanizado con superpoderes deba serlo.

Sin embargo, ambos concuerdan en es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para iniciar este laboratorio, cargue los dataset señalados y visualice a través de la función `head` los atributos que poseen cada uno de los dataset.


In [2]:
# Usar solamente si utilizan Collab.
from google.colab import drive
drive.mount("/content/drive")
dir = '//content//drive//My Drive//Otoño 2021//Laboratorio//Lab 4//'

Mounted at /content/drive


In [3]:
df_comics = pd.read_csv(dir+'df_comics.csv')
df_comics_no_label = pd.read_csv(dir+'comics_no_label.csv')

## 1.1 Obtención de Features [2 puntos]

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Su primera tarea consiste en generar un vector de características para el atributo `history_text`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset (si un personaje tiene este atributo nulo, elimínelo). Luego, para obtener características de texto aplique el modelo de conteo `bag of words` de la siguiente forma:

- Utilice `CountVectorized` junto al tokenizador (que le proveemos) `LemmaTokenizer`.
- Obtenga caracteristicas de los 1-gramas y 2-gramas del texto (ver clase).
- Fijar un maximo de 10.000 caracteristicas para el vector de salida.

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de bag of words caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar MinMaxScaler sobre los atributos de interes.

In [4]:
#### Código aquí ####
df_comics = df_comics.dropna(subset=['history_text'])

In [5]:
stop_words = stopwords.words('english')

class LemmaTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(),
                             lowercase=True,
                             max_features=10000,
                             strip_accents="ascii")

In [10]:
bow = vectorizer.fit_transform(df_comics["history_text"])

In [11]:
print(vectorizer.get_feature_names())

['!', '#', '$', '%', '&', "'", "''", "'d", "'it", "'ll", "'m", "'other-realm", "'power", "'re", "'real", "'s", "'the", "'ve", '(', ')', ',', '-', '--', '-sign-', '.', '...', '/', '/b', '/h', '/i', '030', '1', '10', '100', '1000', '11', '12', '1200', '12th', '13', '14', '142', '15', '16', '17', '18', '19', '1920', '1930', '1936', '1938', '1940', '1940.', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1949', '1950', '1953', '1954', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1970', '1972', '1973', '1975', '1976', '1977', '198', '1980', '1983', '1984', '1985', '1990', '1991', '1992', '1995', '1997', '1999', '19th', '1st', '2', '2.', '20', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2011', '2012', '2014', '2015', '2016', '2017', '2018', '2099', '20th', '21', '21st', '22', '23', '24', '25', '25th', '26', '2nd', '3', '3,000', '3.', '30', '3000', '30th', '31st', '34', '4', '4.', '40', '42', '5', '5,000', '50', '52', '6', '60', '616', '64th', '7', '70', '8',

In [ ]:
from sklearn.preprocessing import MinMaxScaler
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
scaler = MinMaxScaler()
scaler.fit(df_comics[atributos_de_interes])
scaler.transform(df_comics[atributos_de_interes])

array([[0.85, 0.3 , 0.6 , 0.6 , 0.4 , 0.7 ],
       [0.8 , 1.  , 0.8 , 1.  , 1.  , 0.8 ],
       [0.8 , 0.5 , 0.55, 0.45, 1.  , 0.55],
       ...,
       [0.95, 0.5 , 1.  , 0.75, 1.  , 0.8 ],
       [0.75, 0.1 , 1.  , 0.3 , 1.  , 0.3 ],
       [0.45, 0.8 , 0.75, 0.95, 0.8 , 0.5 ]])

In [ ]:
ColumnTransformer(
    transformers=[
                    ('bow', CountVectorizer(
                    tokenizer=LemmaTokenizer(),
                    lowercase=True,  # Transformamos todo a minúsuculas.
                    max_features=10000,  # Dejamos solo las 10000 palabras más frecuentes,
                    ngram_range=(1, 2)
                    ), 
                    'history_text'),
                    ('scaler', MinMaxScaler(), 
                    ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score'])
                  ]
)

## 1.2 Diseño de Pipeline y  Primer Entrenamiento [1.5 puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


A continuación, genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, añadiendo un reductor de dimensionalidad llamado `TruncatedSVD()` ajustando el número de componentes en 1000 (este reducto de dimensionalidad es similar al PCA pero funciona para vectores dispersos) y un clasificador `MultinomialNB()` por defecto.  Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde el etiquetado vendrá dado por el atributo `alignment`. Finalmente entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

**Nota:** Debido al desbalance que existe entre las clases, puede ser util aplicar método de [`imbalanced-learn`](https://github.com/scikit-learn-contrib/imbalanced-learn) como RandomOverSampler sobre los datos de entrenamiento. 

**To-DO:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1,aplicar un reductor de dimensionalidad `TruncatedSVD` y aplicar un clasificador  `MultinomialNB()`.
- [ ] Entrenar el pipeline.
- [ ] (Opcional - **0.5 bonus**) Utilizar técnicas de Sampling para balancear los datos de entrenamiento.


In [39]:
X = df_comics[['history_text','intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']]
y = df_comics['alignment']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.30, random_state=12)

In [45]:
# Balanceamos clases.
over = RandomOverSampler(random_state=1)
X_train_balanced, y_train_balanced = over.fit_resample(X_train, y_train)
X_train_balanced = pd.DataFrame(X_train_balanced)
y_train_balanced = pd.Series(y_train_balanced)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [52]:
index_ = ['history_text','intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
X_train_balanced.columns = index_

In [54]:
# Aca vemos que esta balanceado
y_train_balanced.value_counts()

Neutral    525
Good       525
Bad        525
dtype: int64

In [32]:
preprocessing_transformer = ColumnTransformer(
    transformers=[
                    ('bow', CountVectorizer(
                    tokenizer=LemmaTokenizer(),
                    lowercase=True,  # Transformamos todo a minúsuculas.
                    max_features=10000,  # Dejamos solo las 10000 palabras más frecuentes,
                    ngram_range=(1, 2)
                    ), 
                    'history_text'),
                    ('scaler', MinMaxScaler(), 
                    ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score'])
                  ]
)

pipe = Pipeline(steps=[
                       ('preprocessing', preprocessing_transformer),
                       ('clf', MultinomialNB() )
                       ]
                )
# Probamos desbalanceado
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.52      0.59      0.55       133
        Good       0.70      0.71      0.71       218
     Neutral       0.08      0.03      0.04        35

    accuracy                           0.61       386
   macro avg       0.43      0.44      0.43       386
weighted avg       0.58      0.61      0.59       386



In [56]:
# Balanceado
pipe.fit(X_train_balanced, y_train_balanced)
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.60      0.56      0.58       133
        Good       0.69      0.79      0.73       218
     Neutral       0.36      0.11      0.17        35

    accuracy                           0.65       386
   macro avg       0.55      0.49      0.50       386
weighted avg       0.63      0.65      0.63       386



Podemos notar que al balancear las clases tenemos mejores resultados, sobretodo en la clase neutral

## 1.3 Entrenamiento con Grid Search [2 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen una búsqueda de grilla de los mejores hiperparámetros utilizando `GridSearchCV`. 

Para esto, se le solicita que defina al menos 3 configuraciones de hiperparámetros e intente obtener mejores resultados que los obtenidos en la sección anterior. 

A continuación, un ejemplo de parametros para GridSearch:

```python
params = [
  # esta es la configuración de una busqueda en particular
  # con el clasificador classificator1.
  # en este caso se entrenará el clasificador 1 con combinaciones de todos los 
  # parámetros de bow__max_features, bow__ngram_range, clf__n_estimators 
  # y se seleccionará la mejor combinación.
  {
  'bow__max_features': [5000, 10000, ...],
  'bow__ngram_range': [(1, 1), (1, 2), (1,3)],
  ...,
  'clf': [classificator1()],
  'clf__n_estimators': [200]
  },
  # esta es la configuración de una busqueda en particular
  # con el clasificador classificator2:
  {'clf': [classificator2()],
   'clf__penalty': ['ovr'],
   'clf__multi_class': ['liblinear']
  },
  # esta es la configuración de una busqueda en particular
  # con el clasificador classificator3:
  {'clf': [classificator3()]
  }
             ]
```

Además, note que puede obtener todos los parámetros configurables de un pipeline invocando sobre este el método `.get_params()`.

**Nota:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda dejar corriendo el código y tomarse un tecito.

In [60]:
MAX_FEATURES = [5000, 10000, 15000]
NGRAM_RANGE = [(1,1), (1,2), (1,3)]
FIT_PRIOR = [True, False]

param_grid = [
    {
        'preprocessing__bow__max_features': MAX_FEATURES,
        'preprocessing__bow__ngram_range': NGRAM_RANGE,
        'clf__fit_prior': FIT_PRIOR
    }
]

grid = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid)
grid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('bow',
                                                                         CountVectorizer(analyzer='word',
                                                                                         binary=False,
                                                                                         decode_error='strict',
                                                                                         dtype=<class 'numpy.int64'>,
                             

In [61]:
grid.cv_results_

{'mean_fit_time': array([11.35334992, 12.9886426 , 15.67488885, 11.53019567, 13.00671043,
        15.66645136, 11.41127267, 13.0250483 , 15.60156465, 11.32597909,
        13.09207902, 15.72181764, 11.31367903, 13.03388176, 15.54173608,
        11.38321776, 13.00598016, 15.61596208]),
 'mean_score_time': array([2.82793999, 2.8926703 , 2.95835223, 2.898067  , 2.89173384,
        2.97508488, 2.82097616, 2.89638553, 2.96557498, 2.84230852,
        2.90539422, 2.99474072, 2.81216578, 2.89034452, 2.96713762,
        2.81964698, 2.89325004, 2.97543402]),
 'mean_test_score': array([0.62736189, 0.62401614, 0.62625078, 0.64404718, 0.64292986,
        0.64848541, 0.65962135, 0.64849783, 0.64962135, 0.61845438,
        0.60735568, 0.61180633, 0.6262694 , 0.62956549, 0.62734947,
        0.64961515, 0.63515208, 0.6451707 ]),
 'param_clf__fit_prior': masked_array(data=[True, True, True, True, True, True, True, True, True,
                    False, False, False, False, False, False, False, False,
   

In [69]:

over = RandomOverSampler(random_state=1)
X_balanced, y_balanced = over.fit_resample(X, y)
X_balanced = pd.DataFrame(X_train_balanced)
y_balanced = pd.Series(y_train_balanced)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

MAX_FEATURES = [5000, 10000, 15000]
NGRAM_RANGE = [(1,1), (1,2), (1,3)]
FIT_PRIOR = [True, False]

param_grid = [
    {
        'preprocessing__bow__max_features': MAX_FEATURES,
        'preprocessing__bow__ngram_range': NGRAM_RANGE,
        'clf__fit_prior': FIT_PRIOR
    },
    {
        'clf': [LogisticRegression()],
        'clf__multi_class': ['multinomial']
    },
    {
        'clf': [DecisionTreeClassifier()],
        'clf__criterion': ['gini', 'entropy']
    }
]

grid_2 = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid)
grid_2.fit(X_balanced, y_balanced)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('bow',
                                                                         CountVectorizer(analyzer='word',
                                                                                         binary=False,
                                                                                         decode_error='strict',
                                                                                         dtype=<class 'numpy.int64'>,
                             

In [75]:
pd.concat([pd.DataFrame(grid_2.cv_results_["params"]),pd.DataFrame(grid_2.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)

,clf__fit_prior,preprocessing__bow__max_features,preprocessing__bow__ngram_range,clf,clf__multi_class,clf__criterion,Accuracy
0,True,5000.0,"(1, 1)",NaN,NaN,NaN,0.770159
1,True,5000.0,"(1, 2)",NaN,NaN,NaN,0.716825
2,True,5000.0,"(1, 3)",NaN,NaN,NaN,0.715556
3,True,10000.0,"(1, 1)",NaN,NaN,NaN,0.831111
4,True,10000.0,"(1, 2)",NaN,NaN,NaN,0.765079
5,True,10000.0,"(1, 3)",NaN,NaN,NaN,0.742222
6,True,15000.0,"(1, 1)",NaN,NaN,NaN,0.840000
7,True,15000.0,"(1, 2)",NaN,NaN,NaN,0.765079
8,True,15000.0,"(1, 3)",NaN,NaN,NaN,0.751111
9,False,5000.0,"(1, 1)",NaN,NaN,NaN,0.770159


In [76]:
MAX_FEATURES = [5000, 10000, 15000]
NGRAM_RANGE = [(1,1), (1,2), (1,3)]

param_grid = [
    {
        'preprocessing__bow__max_features': MAX_FEATURES,
        'preprocessing__bow__ngram_range': NGRAM_RANGE,
        'clf': [LogisticRegression()],
        'clf__multi_class': ['multinomial']
    }
]

grid_3 = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid)
grid_3.fit(X_balanced, y_balanced)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('bow',
                                                                         CountVectorizer(analyzer='word',
                                                                                         binary=False,
                                                                                         decode_error='strict',
                                                                                         dtype=<class 'numpy.int64'>,
                             

In [77]:
pd.concat([pd.DataFrame(grid_3.cv_results_["params"]),pd.DataFrame(grid_3.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)

,clf,clf__multi_class,preprocessing__bow__max_features,preprocessing__bow__ngram_range,Accuracy
0,"LogisticRegression(C=1.0, class_weight=None, d...",multinomial,5000,"(1, 1)",0.871111
1,"LogisticRegression(C=1.0, class_weight=None, d...",multinomial,5000,"(1, 2)",0.873016
2,"LogisticRegression(C=1.0, class_weight=None, d...",multinomial,5000,"(1, 3)",0.866667
3,"LogisticRegression(C=1.0, class_weight=None, d...",multinomial,10000,"(1, 1)",0.874286
4,"LogisticRegression(C=1.0, class_weight=None, d...",multinomial,10000,"(1, 2)",0.885714
5,"LogisticRegression(C=1.0, class_weight=None, d...",multinomial,10000,"(1, 3)",0.883175
6,"LogisticRegression(C=1.0, class_weight=None, d...",multinomial,15000,"(1, 1)",0.869841
7,"LogisticRegression(C=1.0, class_weight=None, d...",multinomial,15000,"(1, 2)",0.880000
8,"LogisticRegression(C=1.0, class_weight=None, d...",multinomial,15000,"(1, 3)",0.881905


In [78]:
grid_3.best_estimator_

Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('bow',
                                                  CountVectorizer(analyzer='word',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.int64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                                  lowercase=True,
                                                                  max_df=1.0,
                             

#### 1.3.1 Mejor configuración [0.5]

Comente cual fue la mejor configuración obtenida por Grid Search y por qué cree que esta fue la mejor.

Podemos observar que en el primer Grid Search que se computo el mejor resultado para el Accuracy se obtuvo con el modelo de clasificación Logit en su versión multiclase, y luego en el segundo Grid Search que se computo utilizando este clasificador, se observa que los mejores parametros para el preprocesamiento de texto son un ngram_range = (1,2) y max_features = 10000.
Nos resulta extraño que el modelo Logit sea el que tiene mejor desempeño, ya que este asume que la distribución de los atributos es gaussiana y esto es un poco contraintuitivo, ya que los artistas al crear un personaje probablemente no piensen en que los personajes distribuyan de forma gaussiana, en el sentido de que tengan atributos similares para cada clase, es decir, un artista no se pone a pensar cuales son los atributos clásicos para un hereo, villano o neutral a la hora de crearlo. Posiblemente Logit sea el que mejor funciona en este caso debido a la forma de los datos, que se encuentran entre 0 y 1 debido a la normalización, lo que puede generar que el output de cada observacion este entre 0 y 1, y sabemos que Logit es un modelo que funciona bastante bien cuando la variable de salida es binaria (0 o 1).

## 1.4 Predicción del datos sin etiquetado

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


Llego el momento de predecir cual es la verdadera alineación de `Batcow`. Para esto, deben escoger el mejor pipeline obtenido en las secciones anteriores y predecir la alineación de todos los datos presentes en `df_comics_no_label`.Luego, anexen las alineaciones obtenidas a su correspondiente columna  del dataframe original (atributo `alignment`) y busquen a los flamantes personajes `Batcow`, `Vergil`, y `Gorilla Girl'. Presente los resultados en un `Dataframe`.

**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

In [98]:
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text'])
df_comics_no_label = df_comics_no_label.drop_duplicates()
X_predict = df_comics_no_label[['history_text','intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']]

In [99]:
preprocessing_transformer = ColumnTransformer(
    transformers=[
                    ('bow', CountVectorizer(
                    tokenizer=LemmaTokenizer(),
                    lowercase=True,  # Transformamos todo a minúsuculas.
                    max_features=10000,  # Dejamos solo las 10000 palabras más frecuentes,
                    ngram_range=(1, 2)
                    ), 
                    'history_text'),
                    ('scaler', MinMaxScaler(), 
                    ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score'])
                  ]
)

pipe_logit = Pipeline(steps=[
                       ('preprocessing', preprocessing_transformer),
                       ('clf', LogisticRegression(multi_class = 'multinomial', max_iter = 1000))
                       ]
                )

pipe_logit.fit(X_balanced, y_balanced)
y_pred = pipe_logit.predict(X_predict)

In [100]:
df_comics_no_label['alignment'] = y_pred

In [101]:
entes_poderosos = ['Batcow', 'Vergil', 'Gorilla Girl']
  
df_comics_no_label[df_comics_no_label['name'].isin(entes_poderosos)]

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,...,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
16,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,10,20,"['Animal Attributes', 'Animal Oriented Powers']","['Milkman Man', 'Red Torpedo', 'Red Volcano']","[""Battlin' Bovine""]",NaN,NaN,DC Comics,Good,NaN,Wayne Manor,[],NaN,NaN,Animal,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/13425.jpg?v=157425...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,529,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,90,35,60,60,45,100,"['Agility', 'Jump', 'Super Strength', 'Transfo...",[],['Gorilla Woman'],NaN,NaN,Marvel Comics,Good,NaN,NaN,[],NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
78,1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,90,100,100,"['Accelerated Healing', 'Agility', 'Duplicatio...",[],"['Gilver, Nelo Angelo, Son of Sparda, The Dark...",NaN,Devil May Cry,Capcom,Good,NaN,NaN,[],"Sparda (father) ,Eva (mother) ,Dante (Brother)...",Male,NaN,6'3 • 191 cm,207 lb • 93 kg,Blue,White,NaN,/pictures2/portraits/10/050/11657.jpg?v=154990...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Se observa que los tres entes poderosos son clasificados como buenos héroes :D.
Aplicando san Google pudimos confirmar que tanto Batcow como Gorilla Girl son efectivamente héroes, pero por otro lado, Vergil al ser un antagonista del juego Devil May Cry probablemente sea villano (o por lo menos no un héroe bueno).


### Wordclouds [Opcional- 0,5] 

Una buena pero informal forma de comunicar los resultados del trabajo con texto es generar Wordclouds. Este tipo de visualizaciones nos informan de forma gráfica cuales son las palabras más frecuentes según el tamaño de estas al ser posicionadas en un lienzo.
<center>
<img alt='Ejemplo de una Wordcloud de Starwars' src='https://amueller.github.io/word_cloud/_images/sphx_glr_a_new_hope_001.png' width=400/>

Ejemplo de una Wordcloud de Starwars

</center>
Dicho esto, como equipo docente nos encantaría conocer cuales son las palabras que caracterizan tanto a heroes como neutrales y a villanos y cuales son sus principales diferencias. Por esta razón, les pedimos como última tarea generar una wordcloud con las historias de cada personaje según cada alineamiento (clase). Pueden ocupar el dataset completo para esto. 


**Nota:** Recuerde eliminar las stopwords. Guías completas para generar las wordclouds, eliminar las stopwords y configurar los parámetros de las nubes creadas pueden ser encontradas en su [documentación oficial](https://amueller.github.io/word_cloud/) y en [datacamp](https://www.datacamp.com/community/tutorials/wordcloud-python).





In [ ]:
#### Wordcloud para heroes ####

In [ ]:
#### Wordcloud para neutrales ####

In [ ]:
#### Wordcloud para villanos ####

Comente las principales diferencias entre las tres wordclouds.
¿Hay palabras que caracterizen a los grupos y que no aparezcan en los otros?

---> Comente aquí <---

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>